In [1]:
%%writefile matrix-multiplication.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <time.h>

#define TILE_WIDTH 16  // Tile width for shared memory optimization

// CUDA Kernel for Matrix Multiplication using Tiling
__global__ void matrixMultiplyCUDA(float *A, float *B, float *C, int M, int N, int P) {
    __shared__ float tileA[TILE_WIDTH][TILE_WIDTH];
    __shared__ float tileB[TILE_WIDTH][TILE_WIDTH];

    int row = blockIdx.y * TILE_WIDTH + threadIdx.y;
    int col = blockIdx.x * TILE_WIDTH + threadIdx.x;
    float sum = 0.0;

    for (int k = 0; k < (N + TILE_WIDTH - 1) / TILE_WIDTH; k++) {
        if (row < M && k * TILE_WIDTH + threadIdx.x < N)
            tileA[threadIdx.y][threadIdx.x] = A[row * N + k * TILE_WIDTH + threadIdx.x];
        else
            tileA[threadIdx.y][threadIdx.x] = 0.0;

        if (col < P && k * TILE_WIDTH + threadIdx.y < N)
            tileB[threadIdx.y][threadIdx.x] = B[(k * TILE_WIDTH + threadIdx.y) * P + col];
        else
            tileB[threadIdx.y][threadIdx.x] = 0.0;

        __syncthreads();

        for (int n = 0; n < TILE_WIDTH; n++) {
            sum += tileA[threadIdx.y][n] * tileB[n][threadIdx.x];
        }
        __syncthreads();
    }

    if (row < M && col < P) {
        C[row * P + col] = sum;
    }
}

// CPU-based Matrix Multiplication
void matrixMultiplyCPU(float *A, float *B, float *C, int M, int N, int P) {
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < P; j++) {
            float sum = 0;
            for (int k = 0; k < N; k++) {
                sum += A[i * N + k] * B[k * P + j];
            }
            C[i * P + j] = sum;
        }
    }
}

// Initialize matrix with random values
void initializeMatrix(float *matrix, int rows, int cols) {
    for (int i = 0; i < rows * cols; i++) {
        matrix[i] = (float)rand() / RAND_MAX;
    }
}

// Helper function to get current time in seconds
double getTime() {
    struct timespec ts;
    clock_gettime(CLOCK_REALTIME, &ts);
    return ts.tv_sec + ts.tv_nsec * 1e-9;
}

int main() {
    int M = 1000, N = 1000, P = 1000; // Matrix dimensions

    // Allocate host memory
    float *h_A = (float *)malloc(M * N * sizeof(float));
    float *h_B = (float *)malloc(N * P * sizeof(float));
    float *h_C_CPU = (float *)malloc(M * P * sizeof(float));
    float *h_C_GPU = (float *)malloc(M * P * sizeof(float));

    // Initialize matrices A and B
    initializeMatrix(h_A, M, N);
    initializeMatrix(h_B, N, P);

    // CPU Matrix Multiplication
    double startCPU = getTime();
    matrixMultiplyCPU(h_A, h_B, h_C_CPU, M, N, P);
    double endCPU = getTime();
    double cpuTime = endCPU - startCPU;

    // Allocate device memory
    float *d_A, *d_B, *d_C;
    cudaMalloc((void **)&d_A, M * N * sizeof(float));
    cudaMalloc((void **)&d_B, N * P * sizeof(float));
    cudaMalloc((void **)&d_C, M * P * sizeof(float));

    // Copy data from host to device
    cudaMemcpy(d_A, h_A, M * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, N * P * sizeof(float), cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH);
    dim3 dimGrid((P + TILE_WIDTH - 1) / TILE_WIDTH, (M + TILE_WIDTH - 1) / TILE_WIDTH);

    // Set up CUDA events for GPU timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Start recording GPU time
    cudaEventRecord(start);
    matrixMultiplyCUDA<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, M, N, P);
    cudaEventRecord(stop);

    // Wait for the kernel to complete and calculate elapsed time
    cudaEventSynchronize(stop);
    float gpuTime = 0;
    cudaEventElapsedTime(&gpuTime, start, stop);
    gpuTime /= 1000; // Convert from milliseconds to seconds

    // Copy result from device to host
    cudaMemcpy(h_C_GPU, d_C, M * P * sizeof(float), cudaMemcpyDeviceToHost);

    // Calculate speedup
    double speedup = cpuTime / gpuTime;

    // Output results
    printf("CPU Execution Time: %f seconds\n", cpuTime);
    printf("GPU Execution Time: %f seconds\n", gpuTime);
    printf("Speedup: %f\n", speedup);

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C_CPU);
    free(h_C_GPU);

    // Destroy CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}


Writing matrix-multiplication.cu


In [2]:
!nvcc matrix-multiplication.cu -o matrix-multiplication

In [3]:
!./matrix-multiplication

CPU Execution Time: 6.744320 seconds
GPU Execution Time: 0.141174 seconds
Speedup: 47.772956
